In [8]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
import numpy as log1p
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures


In [9]:
df=pd.read_csv(r"C:\Users\91957\Desktop\python new\hello\30-days-of-ml (1)\train_folds.csv")
df_test= pd.read_csv(r"C:\Users\91957\Desktop\python new\hello\30-days-of-ml (1)\test.csv")
sample_submission=pd.read_csv(r"C:\Users\91957\Desktop\python new\hello\30-days-of-ml (1)\sample_submission.csv")

useful_features=[c for c in df.columns if c not in("id","target","kfold")]
object_cols= [col for col in useful_features if "cat"in col]

df_test= df_test[useful_features]


####
for col in object_cols:
    temp_df=[]
    temp_test_feat=None
    for fold in range(5):
        xtrain= df[df.kfold != fold].reset_index(drop=True)
        xvalid= df[df.kfold== fold].reset_index(drop= True)
        feat= xtrain.groupby(col)["target"].agg('count')
        feat= feat.to_dict()
        
        xvalid.loc[:, f"tar_enc_{col}"]= xvalid[col].map(feat)
        temp_df.append(xvalid)
        if temp_test_feat is None:
            temp_test_feat=df_test[col].map(feat)
        else:
             temp_test_feat=df_test[col].map(feat)
    temp_test_feat/=5
    df_test.loc[:, f"tar_enc_{col}"]= temp_test_feat
    df=pd.concat(temp_df)
      
        

In [10]:
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,tar_enc_cat0,tar_enc_cat1,tar_enc_cat2,tar_enc_cat3,tar_enc_cat4,tar_enc_cat5,tar_enc_cat6,tar_enc_cat7,tar_enc_cat8,tar_enc_cat9
0,1,B,B,B,C,B,B,A,E,C,...,85503,116053,36821,210683,235786,119472,232429,220887,88983,13413
1,8,B,A,A,A,B,D,A,E,C,...,85503,123947,203179,25363,235786,100832,232429,220887,88983,56967
2,13,A,B,A,C,B,B,A,E,A,...,154497,116053,203179,210683,235786,119472,232429,220887,61343,13413
3,14,B,B,A,C,B,D,A,E,C,...,85503,116053,203179,210683,235786,100832,232429,220887,88983,22600
4,25,B,B,A,C,B,D,A,E,C,...,85503,116053,203179,210683,235786,100832,232429,220887,88983,56967


In [11]:
useful_features=[c for c in df.columns if c not in("id","target","kfold")]
object_cols= [col for col in useful_features if col.startswith("cat") ]

df_test= df_test[useful_features]

In [12]:
df.describe()

,id,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,...,tar_enc_cat0,tar_enc_cat1,tar_enc_cat2,tar_enc_cat3,tar_enc_cat4,tar_enc_cat5,tar_enc_cat6,tar_enc_cat7,tar_enc_cat8,tar_enc_cat9
count,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,...,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000
mean,250018.576947,0.527335,0.460926,0.490498,0.496689,0.491654,0.510526,0.467476,0.537119,0.498456,...,129920.897693,120123.944653,177558.614307,187685.293813,231685.777287,103043.081660,225231.694947,203794.386380,67452.071060,31004.564167
std,144450.150010,0.230599,0.214003,0.253346,0.219199,0.240074,0.228232,0.210331,0.218140,0.239920,...,33047.087898,3857.865800,59950.815399,61715.164385,30710.809318,28049.170876,39713.666082,57834.642432,21410.531055,19574.509944
min,1.000000,-0.118039,-0.069309,-0.056104,0.130676,0.255908,0.045915,-0.224689,0.203763,-0.260275,...,85386.000000,116053.000000,36754.000000,461.000000,399.000000,3372.000000,10.000000,14.000000,147.000000,822.000000
25%,124772.500000,0.405965,0.310494,0.300604,0.329783,0.284188,0.354141,0.342873,0.355825,0.332486,...,85509.000000,116121.000000,202962.000000,210630.000000,235773.000000,100832.000000,232373.000000,220701.000000,61260.000000,13353.000000
50%,250002.500000,0.497053,0.427903,0.502462,0.465026,0.390470,0.488865,0.429383,0.504661,0.439151,...,154491.000000,123742.000000,203106.000000,210683.000000,235786.000000,101024.000000,232426.000000,220838.000000,63858.000000,22600.000000
75%,375226.500000,0.668060,0.615113,0.647512,0.664451,0.696599,0.669625,0.573383,0.703441,0.606056,...,154516.000000,123879.000000,203179.000000,210704.000000,235806.000000,119466.000000,232429.000000,220887.000000,88808.000000,47416.000000
max,499999.000000,1.058443,0.887253,1.034704,1.039560,1.055424,1.067649,1.111552,1.032837,1.040229,...,154614.000000,123947.000000,203246.000000,210734.000000,235811.000000,119561.000000,232448.000000,220915.000000,88983.000000,57161.000000


In [13]:
final_predictions=[]
scores=[]
for fold in range(5):
    xtrain= df[df.kfold != fold].reset_index(drop=True)
    xvalid= df[df.kfold== fold].reset_index(drop= True)
    xtest= df_test.copy()
    ytrain= xtrain.target
    yvalid= xvalid.target
    
    xtrain= xtrain[useful_features]
    xvalid= xvalid[useful_features]
    
    
    ordinal_encoder= preprocessing.OrdinalEncoder()
    xtrain[object_cols]= ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols]= ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols]= ordinal_encoder.transform(xtest[object_cols])
    
   
    
    
    model= XGBRegressor(random_state=fold,tree_method='gpu_hist',gpu_id=0, predictor="gpu_predictor")
    model.fit(xtrain, ytrain)
    preds_valid= model.predict(xvalid)
    test_preds=model.predict(xtest)
    final_predictions.append(test_preds)
    rmse= mean_squared_error(yvalid, preds_valid)
    print(fold,rmse)
    scores.append(rmse)
print(np.mean(scores), np.std(scores))

0 0.5258691148263047
1 0.5262090471383132
2 0.530510856197386
3 0.5309427944201653
4 0.5280239755985312
0.5283111576361401 0.0021085113742282803


In [14]:
df.shape

(300000, 37)

In [15]:
sample_submission.shape

(200000, 2)

In [16]:
final_predictions[0]

array([8.15666  , 8.276765 , 8.290124 , ..., 8.288557 , 8.191425 ,
       7.9826827], dtype=float32)

In [17]:
sample_submission["target"] = final_predictions[0]

In [ ]:
sample_submission.to_csv('submission_kaggle.csv', index=False)